# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 9.2481e-01   valLoss:9.4285e-01  time: 6.09e+00
epoch: 1   trainLoss: 8.5345e-01   valLoss:9.4120e-01  time: 1.05e+00
epoch: 2   trainLoss: 8.4340e-01   valLoss:9.3956e-01  time: 1.02e+00
epoch: 3   trainLoss: 7.8019e-01   valLoss:9.4302e-01  time: 1.00e+00
epoch: 4   trainLoss: 7.6279e-01   valLoss:9.3952e-01  time: 1.02e+00
epoch: 5   trainLoss: 6.7115e-01   valLoss:9.3978e-01  time: 1.01e+00
epoch: 6   trainLoss: 6.8930e-01   valLoss:8.9587e-01  time: 1.01e+00
epoch: 7   trainLoss: 6.7734e-01   valLoss:8.3574e-01  time: 1.01e+00
epoch: 8   trainLoss: 6.0780e-01   valLoss:7.1396e-01  time: 1.01e+00
epoch: 9   trainLoss: 5.9675e-01   valLoss:6.2857e-01  time: 1.01e+00
epoch: 10   trainLoss: 5.5870e-01   valLoss:5.8399e-01  time: 9.95e-01
epoch: 11   trainLoss: 5.5069e-01   valLoss:5.6434e-01  time: 9.95e-01
epoch: 12   trainLoss: 4.7988e-01   valLoss:5.4922e-01  time: 1.02e+00
epoch: 13   trainLoss: 4.8973e-01   valLoss:5.3096e-01  time: 1.03e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000024,0.003276,0.145015,0.819902,0.917718,0.278150,-11.358551
test,0.000024,0.003225,0.145648,0.740120,0.889778,0.366591,-8.470625


## 5. Visualize some predictions

In [10]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

alt.HConcatChart(...)